# Multi-Layer Perceptron

In [1]:
import random
import numpy as np
class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,lamda,test_data=None):
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta,lamda,len(training_data))
            if test_data:
                print ("Epoch", j,", Training Error:", self.evaluate(test_data),"|",n_test)
            else:
                print("Done")

    def update_mini_batch(self, mini_batch, eta,lamda,n):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*lamda/n)*w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)
    
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [2]:
import cloudpickle as cPickle
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('datahw3/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [3]:
training_data, validation_data, test_data = load_data_wrapper()

net = Network([784, 40,10, 10])
net.SGD(training_data, 30, 10, 3.0,0,test_data=validation_data)
print("Test Error:",net.evaluate(test_data),"|",len(test_data))

Epoch 0 , Training Error: 9020 | 10000
Epoch 1 , Training Error: 9209 | 10000
Epoch 2 , Training Error: 9274 | 10000
Epoch 3 , Training Error: 9348 | 10000
Epoch 4 , Training Error: 9372 | 10000
Epoch 5 , Training Error: 9345 | 10000
Epoch 6 , Training Error: 9378 | 10000
Epoch 7 , Training Error: 9444 | 10000
Epoch 8 , Training Error: 9437 | 10000
Epoch 9 , Training Error: 9457 | 10000
Epoch 10 , Training Error: 9495 | 10000
Epoch 11 , Training Error: 9445 | 10000
Epoch 12 , Training Error: 9501 | 10000
Epoch 13 , Training Error: 9502 | 10000
Epoch 14 , Training Error: 9491 | 10000
Epoch 15 , Training Error: 9464 | 10000
Epoch 16 , Training Error: 9512 | 10000
Epoch 17 , Training Error: 9504 | 10000
Epoch 18 , Training Error: 9527 | 10000
Epoch 19 , Training Error: 9455 | 10000
Epoch 20 , Training Error: 9495 | 10000
Epoch 21 , Training Error: 9524 | 10000
Epoch 22 , Training Error: 9482 | 10000
Epoch 23 , Training Error: 9537 | 10000
Epoch 24 , Training Error: 9525 | 10000
Epoch 25 ,

In [4]:
net.SGD(training_data, 30, 10, 3.0,1,test_data=validation_data)
print("Test Error:",net.evaluate(test_data),"|",len(test_data))

Epoch 0 , Training Error: 9569 | 10000
Epoch 1 , Training Error: 9571 | 10000
Epoch 2 , Training Error: 9586 | 10000
Epoch 3 , Training Error: 9570 | 10000
Epoch 4 , Training Error: 9650 | 10000
Epoch 5 , Training Error: 9552 | 10000
Epoch 6 , Training Error: 9408 | 10000
Epoch 7 , Training Error: 9629 | 10000
Epoch 8 , Training Error: 9604 | 10000
Epoch 9 , Training Error: 9559 | 10000
Epoch 10 , Training Error: 9555 | 10000
Epoch 11 , Training Error: 9594 | 10000
Epoch 12 , Training Error: 9597 | 10000
Epoch 13 , Training Error: 9623 | 10000
Epoch 14 , Training Error: 9606 | 10000
Epoch 15 , Training Error: 9656 | 10000
Epoch 16 , Training Error: 9590 | 10000
Epoch 17 , Training Error: 9614 | 10000
Epoch 18 , Training Error: 9602 | 10000
Epoch 19 , Training Error: 9608 | 10000
Epoch 20 , Training Error: 9604 | 10000
Epoch 21 , Training Error: 9605 | 10000
Epoch 22 , Training Error: 9599 | 10000
Epoch 23 , Training Error: 9526 | 10000
Epoch 24 , Training Error: 9538 | 10000
Epoch 25 ,

In [5]:
net = Network([784, 150,85,15, 10])
net.SGD(training_data, 30, 10, 3.0,0,test_data=test_data)
print("Test Error:",net.evaluate(test_data),"|",len(test_data))

Epoch 0 , Training Error: 8962 | 10000
Epoch 1 , Training Error: 9074 | 10000
Epoch 2 , Training Error: 9253 | 10000
Epoch 3 , Training Error: 9295 | 10000
Epoch 4 , Training Error: 9271 | 10000
Epoch 5 , Training Error: 9409 | 10000
Epoch 6 , Training Error: 9374 | 10000
Epoch 7 , Training Error: 9354 | 10000
Epoch 8 , Training Error: 9419 | 10000
Epoch 9 , Training Error: 9485 | 10000
Epoch 10 , Training Error: 9479 | 10000
Epoch 11 , Training Error: 9498 | 10000
Epoch 12 , Training Error: 9493 | 10000
Epoch 13 , Training Error: 9547 | 10000
Epoch 14 , Training Error: 9595 | 10000
Epoch 15 , Training Error: 9534 | 10000
Epoch 16 , Training Error: 9588 | 10000
Epoch 17 , Training Error: 9582 | 10000
Epoch 18 , Training Error: 9587 | 10000
Epoch 19 , Training Error: 9516 | 10000
Epoch 20 , Training Error: 9579 | 10000
Epoch 21 , Training Error: 9585 | 10000
Epoch 22 , Training Error: 9614 | 10000
Epoch 23 , Training Error: 9590 | 10000
Epoch 24 , Training Error: 9550 | 10000
Epoch 25 ,

In [6]:
net.SGD(training_data, 30, 10, 3.0,1,test_data=test_data)
print("Test Error:",net.evaluate(test_data),"|",len(test_data))

Epoch 0 , Training Error: 9639 | 10000
Epoch 1 , Training Error: 9665 | 10000
Epoch 2 , Training Error: 9654 | 10000
Epoch 3 , Training Error: 9599 | 10000
Epoch 4 , Training Error: 9611 | 10000
Epoch 5 , Training Error: 9642 | 10000
Epoch 6 , Training Error: 9587 | 10000
Epoch 7 , Training Error: 9647 | 10000
Epoch 8 , Training Error: 9710 | 10000
Epoch 9 , Training Error: 9653 | 10000
Epoch 10 , Training Error: 9670 | 10000
Epoch 11 , Training Error: 9681 | 10000
Epoch 12 , Training Error: 9697 | 10000
Epoch 13 , Training Error: 9674 | 10000
Epoch 14 , Training Error: 9584 | 10000
Epoch 15 , Training Error: 9473 | 10000
Epoch 16 , Training Error: 9723 | 10000
Epoch 17 , Training Error: 9664 | 10000
Epoch 18 , Training Error: 9711 | 10000
Epoch 19 , Training Error: 9643 | 10000
Epoch 20 , Training Error: 9741 | 10000
Epoch 21 , Training Error: 9729 | 10000
Epoch 22 , Training Error: 9579 | 10000
Epoch 23 , Training Error: 9726 | 10000
Epoch 24 , Training Error: 9758 | 10000
Epoch 25 ,